# Srock Strategy - Signal Event 0002 - Indicator Timing Strategy

## 1. Import Library

In [6]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [7]:
sd = '2015-12-31'
ed = '2026-02-13'

In [8]:
instrument_list = ['600519.SH']

In [9]:
sql = f"""
SELECT
    date,
    instrument,
    m_ta_macd_dea(close) AS DEA,
    m_ta_macd_dif(close) AS DIF,
    IF(DIF > DEA AND m_lag(DIF, 1) < m_lag(DEA, 1), 1, 0) AS signal_buy,
    IF(DIF < DEA AND m_lag(DIF, 1) > m_lag(DEA, 1), 1, 0) AS signal_sell,
FROM cn_stock_bar1d
WHERE instrument IN {str(tuple(instrument_list))}
QUALIFY (signal_buy = 1 OR signal_sell = 1)
"""

In [10]:
df = dai.query(sql, filters = {"date":[sd, ed]}).df()
df

,date,instrument,DEA,DIF,signal_buy,signal_sell
0,2024-07-02,600519.SH,-329.325945,-326.436803,1,0
1,2024-07-26,600519.SH,-153.917515,-160.361749,0,1
2,2024-08-08,600519.SH,-208.567357,-189.597793,1,0
3,2024-09-09,600519.SH,-83.299550,-92.257530,0,1
4,2024-09-25,600519.SH,-228.229014,-207.500439,1,0
...,...,...,...,...,...,...
187,2024-03-04,600519.SH,84.598458,82.362771,0,1
188,2024-03-12,600519.SH,74.137756,89.810627,1,0
189,2024-03-20,600519.SH,93.782143,93.245412,0,1
190,2024-04-23,600519.SH,-54.380347,-49.851635,1,0


## 3. Backtest Strategy

In [11]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 1
    context.is_trade_by_weight = True
    context.weight = 1 / len(context.data['instrument'].unique())

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return

    instruments_buy  = set(df_now[df_now["signal_buy"]  == 1]["instrument"])
    instruments_sell = set(df_now[df_now["signal_sell"] == 1]["instrument"])
    instruments_hold = set(context.get_account_positions().keys())

    for instrument in instruments_buy - instruments_hold:
        if context.is_trade_by_weight:
            context.order_target_percent(instrument, context.weight)
        else:
            context.order(instrument, 100)
    
    for instrument in instruments_sell:
        context.order_target_percent(instrument, 0)

    for instrument in instruments_hold:
        if (data.current_dt - context.get_position(instrument).last_sale_date).days >= context.holding_days:
            context.order_target_percent(instrument, 0)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 100000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 14:33:53] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 14:33:53] [info     ] 2016-03-17, 2026-01-29, , , instruments=1
[2026-02-18 14:33:53] [info     ] bigtrader module V2.2.0
[2026-02-18 14:33:53] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 14:33:56] [info     ] backtest done, raw_perf_ds:dai.DataSource("_c575bd8338d14852bbb155e3da925c1b")


[2026-02-18 14:33:57] [info     ] bigtrader.v34 运行完成 [4.036s].
